# Lab 4: SVM + Neural Networks #


In [2]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [48]:
df_train = pd.read_csv('./lab_4_training.csv')
df_test = pd.read_csv('./lab_4_test.csv')
df_train.head()

,Unnamed: 0,gender,age,year,eyecolor,height,miles,brothers,sisters,computertime,exercise,exercisehours,musiccds,playgames,watchtv
0,1303,male,20,second,green,73.0,210.0,0,1,10.0,Yes,5.0,50.0,1.0,15.0
1,36,male,20,third,other,71.0,90.0,1,0,15.0,Yes,4.0,10.0,0.0,1.0
2,489,male,22,fourth,hazel,75.0,200.0,0,1,1.0,Yes,2.0,150.0,1.0,10.0
3,1415,male,19,second,brown,72.0,35.0,2,2,20.0,Yes,5.0,100.0,0.0,7.0
4,616,male,22,fourth,hazel,71.0,15.0,2,1,10.0,Yes,7.0,10.0,0.0,5.0


***
### Question 1###
Calculate a baseline accuracy measure using the majority class.

** Question 1.a**  
Find the majority class in the training set. If you always predicted this class in the training set, what would your accuracy be?

In [4]:


print(df_train.groupby('gender').size())
print()
print('Q 1A ::','The majority class of samples in the training set is "female".')

class_sizes = list(df_train.groupby('gender').size())
accuracy_majority_class = np.max(class_sizes) / np.sum(class_sizes)
print('Q 1A ::','Fraction (=Accuracy) of teh majority class in the training set is ', accuracy_majority_class)

gender
female    855
male      735
dtype: int64

Q 1A :: The majority class of samples in the training set is "female".
Q 1A :: Fraction (=Accuracy) of teh majority class in the training set is  0.5377358490566038


**Question 1.b**   
If you always predicted this same class (majority from the training set) in the test set, what would your accuracy be?

In [5]:
print(df_test.groupby('gender').size())
print()
print('Q 1B ::','The majority class of samples in the training set is "female".')

class_sizes = list(df_test.groupby('gender').size())
accuracy_majority_class = np.max(class_sizes) / np.sum(class_sizes)
print('Q 1B ::','Fraction (=Accuracy) of teh majority class in the training set is ', accuracy_majority_class)

gender
female    208
male      190
dtype: int64

Q 1B :: The majority class of samples in the training set is "female".
Q 1B :: Fraction (=Accuracy) of teh majority class in the training set is  0.5226130653266332


***
### Question 2 ###
Get started with Neural Networks.

   
Choose a NN implementation and specify which you choose. Be sure the implementation allows you to modify the number of hidden layers and hidden nodes per layer.  

NOTE: When possible, specify the logsig (sigmoid/logistc) function as the transfer function for the output node and use Levenberg-Marquardt backpropagation (lbfgs). It is possible to specify logsig or logistic in Sklearn MLPclassifier (Neural net).  

**Question 2.a**   
Train a neural network with a single 10 node hidden layer. Only use the Height feature of the dataset to predict the Gender. You will have to change Gender to a 0 and 1 class. After training, use your trained model to predict the class using the height feature from the training set. What was the accuracy of this prediction?

In [6]:
# KERAS
from keras.models import Sequential
from keras.layers import Dense, Activation

df_train_reduced = df_train[['gender', 'height']]
df_train_reduced['gender'] = df_train_reduced['gender'].apply(lambda x : 1 if x == 'male' else 0)

model = Sequential([
    Dense(10, activation='sigmoid', input_shape=(1,)),
    Dense(1,),
])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x=df_train_reduced['height'], y=df_train_reduced['gender'])

evaluation = model.evaluate(x=df_train_reduced['height'], y=df_train_reduced['gender'])
# print(model.metrics_names)
print('Q 1B ::','Accuracy (Train Set) ', evaluation[1])

Using TensorFlow backend.


Epoch 1/1
1590/1590 [==============================] - 0s 33us/step
Q 1B :: Accuracy (Train Set)  0.5377358486817317


**Question 2.b**  
Take the trained model from question 2.b and use it to predict the test set. This can be accomplished by taking the trained model and giving it the Height feature values from the test set. What is the accuracy of this model on the test set?

In [8]:
y_test = df_test['gender'].apply(lambda x: 0 if x == 'male' else 1).values.reshape(-1,1)
x_test= df_test['height'].values.reshape(-1,1)

evaluation = model.evaluate(x_test, y_test, batch_size=256)

print('Q 2B ::','Accuracy (Test Set) ', evaluation[1])

398/398 [==============================] - 0s 12us/step
Q 2B :: Accuracy (Test Set)  0.4773869355719293


**Question 2.c**   

Neural Networks tend to prefer smaller, normalized feature values. Try taking the log of the height feature in both training and testing sets or use a Standard Scalar operation in SKlearn to centre and normalize the data between 0-1 for continuous values. Repeat question 2.c with the log version and the normalized and centered version of this feature

In [12]:
y_test = df_test['gender'].apply(lambda x: 0 if x == 'male' else 1).values.reshape(-1,1)
y_train = df_train['gender'].apply(lambda x: 0 if x == 'male' else 1).values.reshape(-1,1)

x_test_normalized = df_test['height'].apply('log').values.reshape(-1,1)
x_train_normalized = df_train['height'].apply('log').values.reshape(-1,1)

model.fit(x_train_normalized, y_train, epochs=2, batch_size=256)
evaluation = model.evaluate(x_test_normalized, y_test, batch_size=256)

print('Q 2C ::','Accuracy (Test Set) ', evaluation[1])
# seems to make not a big difference at all

Epoch 1/2
1590/1590 [==============================] - 0s 8us/step - loss: 8.6673 - acc: 0.4623
Epoch 2/2
398/398 [==============================] - 0s 14us/step
Q 2C :: Accuracy (Test Set)  0.4773869355719293


***
### Question 3 ###
Get started with Support Vector Machines.

  
Chose a SVM implementation and specify which you choose. Be sure the implementation allows you to choose between linear and RBF kernels.

**Question 3.a**   
Use the same dataset from 2.a using the linear kernel to find training set prediction accuracy.

In [17]:
from sklearn import svm

y_train = df_train['gender'].apply(lambda x: 0 if x == 'male' else 1).values.reshape(-1,1)
x_train = df_train['height'].values.reshape(-1,1)
y_test = df_test['gender'].apply(lambda x: 0 if x == 'male' else 1).values.reshape(-1,1)
x_test = df_test['height'].values.reshape(-1,1)

clf = svm.SVC(kernel='linear')
clf.fit(x_train, y_train)

accuracy = clf.score(x_train, y_train)

print('Q 3A ::','Accuracy SVM (Train Set) ', accuracy)

Q 3A :: Accuracy SVM (Train Set)  0.8465408805031447


**Question 3.b**   
Use the same dataset from 2.a using the linear kernel to find test set prediction accuracy

In [18]:
accuracy = clf.score(x_test, y_test)

print('Q 3B ::','Accuracy SVM (Test Set) ', accuracy)

Q 3B :: Accuracy SVM (Test Set)  0.8542713567839196


**Question 3.c**   
Use the same dataset from 2.a using the RBF kernel  to find training set prediction accuracy

In [20]:
clf = svm.SVC(kernel='rbf')
clf.fit(x_train, y_train)

accuracy = clf.score(x_train, y_train)

print('Q 3C ::','Accuracy SVM with rbf kernel (Train Set) ', accuracy)

Q 3C :: Accuracy SVM with rbf kernel (Train Set)  0.8465408805031447


**Question 3.d**   
Use the same dataset from 2.a using the RBF kernel  to find test set prediction accuracy

In [21]:
accuracy = clf.score(x_test, y_test)

print('Q 3D ::','Accuracy SVM with rbf kernel (Test Set) ', accuracy)

Q 3D :: Accuracy SVM with rbf kernel (Test Set)  0.8542713567839196


**Question 3.e**   
Use the same dataset from 2.c (log) using the RBF to find test set prediction accuracy

In [22]:
clf = svm.SVC(kernel='rbf')
x_train = df_train['height'].apply('log').values.reshape(-1,1)
x_test = df_test['height'].apply('log').values.reshape(-1,1)
clf.fit(x_train, y_train)

accuracy = clf.score(x_test, y_test)

print('Q 3E ::','Accuracy SVM with rbf kernel and log (Test Set) ', accuracy)

Q 3E :: Accuracy SVM with rbf kernel and log (Test Set)  0.8542713567839196


**Question 3.f**   
Z-score is a normalization technique. It is the value of a feature minus the average value for that feature in the training set, divided by the standard deviation of that feature in the training set. Repeat question 3.e using Z-score and note if there is any difference in accuracy and comment on why there is a change or no change in accuracy

In [23]:
avg = df_train['height'].mean()
std_deviation = df_train['height'].std()

transform_z_score = lambda x: (x - avg) / std_deviation

x_train = df_train['height'].apply(transform_z_score).values.reshape(-1,1)
x_test = df_test['height'].apply(transform_z_score).values.reshape(-1,1)

clf.fit(x_train, y_train)

accuracy = clf.score(x_test, y_test)
print('Q 3E ::','Accuracy SVM with rbf kernel and z-score (Test Set) ', accuracy)

Q 3E :: Accuracy SVM with rbf kernel and z-score (Test Set)  0.8542713567839196


***

### Question 4 ###
The rest of features in this dataset barring a few are categorical. Neither ML method accepts categorical features, so transform year, eyecolor, exercise into a set of binary features, one feature per unique original feature value, and mark the binary feature as ‘1’ if the feature value matches the original value and ‘0’ otherwise. Using only these binary variable transformed features, train and predict the class of the test set.

In [24]:
df_train4 = df_train[['year', 'eyecolor', 'exercise']]
df_test4 = df_test[['year', 'eyecolor', 'exercise']]

X_train = df_train4.to_dict('records')
y_train = df_train['gender']

X_test = df_test4.to_dict('records')
y_test = df_test['gender']

vec = DictVectorizer()
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.fit_transform(X_test).toarray()

le = LabelEncoder()
le.fit(["male", "female"])
y_train = le.transform(y_train).reshape(-1,1)
y_test = le.transform(y_test).reshape(-1,1)

**Question 4.a**    
What was your accuracy using Neural Network with a single 10 node hidden layer? During training, use a maximum number of iterations of 50. (Expected training time: ~15 mins)

In [25]:
model = Sequential()
model.add(Dense(units=10, activation='sigmoid', input_dim=13))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
1590/1590 [==============================] - 0s 256us/step - loss: 0.6918 - acc: 0.5296
Epoch 2/50
1590/1590 [==============================] - 0s 57us/step - loss: 0.6917 - acc: 0.5346
Epoch 3/50
1590/1590 [==============================] - 0s 60us/step - loss: 0.6916 - acc: 0.5239
Epoch 4/50
1590/1590 [==============================] - 0s 51us/step - loss: 0.6916 - acc: 0.5296
Epoch 5/50
1590/1590 [==============================] - 0s 63us/step - loss: 0.6914 - acc: 0.5289
Epoch 6/50
1590/1590 [==============================] - 0s 49us/step - loss: 0.6914 - acc: 0.5270
Epoch 7/50
1590/1590 [==============================] - 0s 51us/step - loss: 0.6913 - acc: 0.5258
Epoch 8/50
1590/1590 [==============================] - 0s 51us/step - loss: 0.6912 - acc: 0.5321
Epoch 9/50
1590/1590 [==============================] - 0s 53us/step - loss: 0.6911 - acc: 0.5270
Epoch 10/50
1590/1590 [==============================] - 0s 63us/step - loss: 0.6910 - acc: 0.5277
Epoch 11/50
1590/1

In [26]:
evaluation = model.evaluate(X_test, y_test)
print('Q 4A ::','Accuracy (Test Set) ', evaluation[1])

398/398 [==============================] - 0s 117us/step
Q 4A :: Accuracy (Test Set)  0.5427135684382376


**Question 4.b**    
What was your accuracy using a SVM with RBF kernel?

In [28]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

accuracy = clf.score(X_test, y_test)
print('Q 4B ::','Accuracy SVM (Test Set) ', accuracy)

Q 4B :: Accuracy SVM (Test Set)  0.585427135678392


***
### Question 5###
Using a NN, does height + eye color predict the test set class better by:

**Question 5.a**  
Keeping the original feature values?

In [30]:
X_train = df_train[['height','eyecolor']]
X_train = X_train.to_dict('records')

X_test = df_test[['height','eyecolor']]
X_test = X_test.to_dict('records')

vec = DictVectorizer()
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.fit_transform(X_test).toarray()

model = Sequential()
model.add(Dense(units=10, activation='sigmoid', input_dim=6))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1590/1590 [==============================] - 0s 243us/step - loss: 0.6909 - acc: 0.5377
Epoch 2/10
1590/1590 [==============================] - 0s 73us/step - loss: 0.6907 - acc: 0.5377
Epoch 3/10
1590/1590 [==============================] - 0s 51us/step - loss: 0.6906 - acc: 0.5377
Epoch 4/10
1590/1590 [==============================] - 0s 57us/step - loss: 0.6905 - acc: 0.5377
Epoch 5/10
1590/1590 [==============================] - 0s 54us/step - loss: 0.6905 - acc: 0.5377
Epoch 6/10
1590/1590 [==============================] - 0s 52us/step - loss: 0.6905 - acc: 0.5377
Epoch 7/10
1590/1590 [==============================] - 0s 60us/step - loss: 0.6905 - acc: 0.5377
Epoch 8/10
1590/1590 [==============================] - 0s 56us/step - loss: 0.6904 - acc: 0.5377
Epoch 9/10
1590/1590 [==============================] - 0s 56us/step - loss: 0.6904 - acc: 0.5377
Epoch 10/10
1590/1590 [==============================] - 0s 62us/step - loss: 0.6904 - acc: 0.5377


In [31]:
evaluation = model.evaluate(X_test, y_test)
print('Q 5A ::','Accuracy (Test Set) ', evaluation[1])

398/398 [==============================] - 0s 154us/step
Q 5A :: Accuracy (Test Set)  0.5226130659256748


**Question 5.b**  
Taking the log of the original values?

In [33]:
df_train_log = df_train
df_test_log = df_test
df_train_log['height'] = df_train['height'].apply('log')
df_test_log['height'] = df_test['height'].apply('log')

X_train = df_train_log[['height','eyecolor']]
X_train = X_train.to_dict('records')

X_test = df_test_log[['height','eyecolor']]
X_test = X_test.to_dict('records')

vec = DictVectorizer()
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.fit_transform(X_test).toarray()

model = Sequential()
model.add(Dense(units=10, activation='sigmoid', input_dim=6))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1590/1590 [==============================] - 0s 313us/step - loss: 0.7008 - acc: 0.4730
Epoch 2/10
1590/1590 [==============================] - 0s 50us/step - loss: 0.6976 - acc: 0.4635
Epoch 3/10
1590/1590 [==============================] - 0s 42us/step - loss: 0.6963 - acc: 0.4818
Epoch 4/10
1590/1590 [==============================] - 0s 40us/step - loss: 0.6958 - acc: 0.5088
Epoch 5/10
1590/1590 [==============================] - 0s 51us/step - loss: 0.6955 - acc: 0.5088
Epoch 6/10
1590/1590 [==============================] - 0s 52us/step - loss: 0.6954 - acc: 0.5088
Epoch 7/10
1590/1590 [==============================] - 0s 49us/step - loss: 0.6953 - acc: 0.5233
Epoch 8/10
1590/1590 [==============================] - 0s 49us/step - loss: 0.6951 - acc: 0.5088
Epoch 9/10
1590/1590 [==============================] - 0s 50us/step - loss: 0.6951 - acc: 0.5107
Epoch 10/10
1590/1590 [==============================] - 0s 54us/step - loss: 0.6950 - acc: 0.5270


In [34]:
evaluation = model.evaluate(X_test, y_test)
print('Q 5B::','Accuracy (Test Set) ', evaluation[1])

398/398 [==============================] - 0s 192us/step
Q 5B:: Accuracy (Test Set)  0.5226130659256748


**Question 5.c**  
Taking the Z-score of the original values?

In [35]:
avg = df_train['height'].mean()
std_deviation = df_train['height'].std()

transform_z_score = lambda x: (x - avg) / std_deviation


df_train_Z = df_train
df_test_Z = df_test
df_train_Z['height'] = df_train_Z['height'].apply(transform_z_score)
df_test_Z['height'] = df_test_Z['height'].apply(transform_z_score)

df_train_Z = df_train_Z[['height','eyecolor']]
X_train = df_train_Z[['height','eyecolor']]
X_train = X_train.to_dict('records')

df_test_Z = df_test_Z[['height','eyecolor']]
X_test = df_test_Z[['height','eyecolor']]
X_test = X_test.to_dict('records')

vec = DictVectorizer()
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.fit_transform(X_test).toarray()

model = Sequential()
model.add(Dense(units=10, activation='sigmoid', input_dim=6))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1590/1590 [==============================] - 0s 209us/step - loss: 0.6603 - acc: 0.5377
Epoch 2/10
1590/1590 [==============================] - 0s 45us/step - loss: 0.6501 - acc: 0.5560
Epoch 3/10
1590/1590 [==============================] - 0s 47us/step - loss: 0.6422 - acc: 0.6220
Epoch 4/10
1590/1590 [==============================] - 0s 48us/step - loss: 0.6353 - acc: 0.6635
Epoch 5/10
1590/1590 [==============================] - 0s 47us/step - loss: 0.6288 - acc: 0.7088
Epoch 6/10
1590/1590 [==============================] - 0s 47us/step - loss: 0.6225 - acc: 0.7239
Epoch 7/10
1590/1590 [==============================] - 0s 50us/step - loss: 0.6164 - acc: 0.7579
Epoch 8/10
1590/1590 [==============================] - 0s 49us/step - loss: 0.6103 - acc: 0.7648
Epoch 9/10
1590/1590 [==============================] - 0s 46us/step - loss: 0.6042 - acc: 0.7679
Epoch 10/10
1590/1590 [==============================] - 0s 46us/step - loss: 0.5981 - acc: 0.7780


In [37]:
evaluation = model.evaluate(X_test, y_test)
print('Q 5C::','Accuracy (Test Set) ', evaluation[1])
# now it improves accuracy

398/398 [==============================] - 0s 36us/step
Q 5C:: Accuracy (Test Set)  0.7663316576924156


***
### Question 6 ###
Repeat question 5 for exercise hours + eye color

In [38]:
X_train = df_train[['exercisehours','eyecolor']]
X_train = X_train.to_dict('records')

X_test = df_test[['exercisehours','eyecolor']]
X_test = X_test.to_dict('records')

vec = DictVectorizer()
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.fit_transform(X_test).toarray()

model = Sequential()
model.add(Dense(units=10, activation='sigmoid', input_dim=6))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)


Epoch 1/10
1590/1590 [==============================] - 0s 212us/step - loss: 0.8752 - acc: 0.4623
Epoch 2/10
1590/1590 [==============================] - 0s 37us/step - loss: 0.7664 - acc: 0.4623
Epoch 3/10
1590/1590 [==============================] - 0s 41us/step - loss: 0.7236 - acc: 0.4623
Epoch 4/10
1590/1590 [==============================] - 0s 41us/step - loss: 0.7068 - acc: 0.4421
Epoch 5/10
1590/1590 [==============================] - 0s 38us/step - loss: 0.7002 - acc: 0.4396
Epoch 6/10
1590/1590 [==============================] - 0s 35us/step - loss: 0.6973 - acc: 0.4862
Epoch 7/10
1590/1590 [==============================] - 0s 41us/step - loss: 0.6957 - acc: 0.5170
Epoch 8/10
1590/1590 [==============================] - 0s 42us/step - loss: 0.6947 - acc: 0.5182
Epoch 9/10
1590/1590 [==============================] - 0s 48us/step - loss: 0.6938 - acc: 0.5195
Epoch 10/10
1590/1590 [==============================] - 0s 50us/step - loss: 0.6931 - acc: 0.5346


In [40]:
evaluation = model.evaluate(X_test, y_test)
print('Q 6A::','Accuracy (Test Set) ', evaluation[1])

398/398 [==============================] - 0s 39us/step
Q 6A:: Accuracy (Test Set)  0.5226130659256748


In [52]:
df_train_log = df_train
df_test_log = df_test


# make sure there are no NaN values in the resulting dataset by replacing 0 with a small positive number
df_train_log['exercisehours'] = df_train_log['exercisehours'].apply(lambda x : 0.00001 if x == 0 else x)
df_test_log['exercisehours'] = df_test_log['exercisehours'].apply(lambda x : 0.00001 if x == 0 else x)

df_train_log['exercisehours'] = df_train['exercisehours'].apply('log')
df_test_log['exercisehours'] = df_test['exercisehours'].apply('log')

df_train_log

X_train = df_train_log[['exercisehours','eyecolor']]
X_train = X_train.to_dict('records')

X_test = df_test_log[['exercisehours','eyecolor']]
X_test = X_test.to_dict('records')

vec = DictVectorizer()
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.fit_transform(X_test).toarray()

model = Sequential()
model.add(Dense(units=10, activation='sigmoid', input_dim=6))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1590/1590 [==============================] - 1s 320us/step - loss: nan - acc: 0.0038
Epoch 2/10
1590/1590 [==============================] - 0s 43us/step - loss: nan - acc: 0.0000e+00
Epoch 3/10
1590/1590 [==============================] - 0s 46us/step - loss: nan - acc: 0.0000e+00
Epoch 4/10
1590/1590 [==============================] - 0s 44us/step - loss: nan - acc: 0.0000e+00
Epoch 5/10
1590/1590 [==============================] - 0s 43us/step - loss: nan - acc: 0.0000e+00
Epoch 6/10
1590/1590 [==============================] - 0s 42us/step - loss: nan - acc: 0.0000e+00
Epoch 7/10
1590/1590 [==============================] - 0s 39us/step - loss: nan - acc: 0.0000e+00
Epoch 8/10
1590/1590 [==============================] - 0s 41us/step - loss: nan - acc: 0.0000e+00
Epoch 9/10
1590/1590 [==============================] - 0s 42us/step - loss: nan - acc: 0.0000e+00
Epoch 10/10
1590/1590 [==============================] - 0s 43us/step - loss: nan - acc: 0.0000e+00


In [53]:
evaluation = model.evaluate(X_test, y_test)
print('Q 6B::','Accuracy (Test Set) ', evaluation[1])

398/398 [==============================] - 0s 285us/step
Q 6B:: Accuracy (Test Set)  0.0


In [54]:
avg = df_train['exercisehours'].mean()
std_deviation = df_train['exercisehours'].std()

transform_z_score = lambda x: (x - avg) / std_deviation


df_train_Z = df_train
df_test_Z = df_test
df_train_Z['exercisehours'] = df_train_Z['exercisehours'].apply(transform_z_score)
df_test_Z['exercisehours'] = df_test_Z['exercisehours'].apply(transform_z_score)

X_train = df_train_Z[['exercisehours','eyecolor']]
X_train = X_train.to_dict('records')

X_test = df_test_Z[['exercisehours','eyecolor']]
X_test = X_test.to_dict('records')

vec = DictVectorizer()
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.fit_transform(X_test).toarray()

model = Sequential()
model.add(Dense(units=10, activation='sigmoid', input_dim=6))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)


Epoch 1/10
1590/1590 [==============================] - 1s 334us/step - loss: nan - acc: 0.0069
Epoch 2/10
1590/1590 [==============================] - 0s 51us/step - loss: nan - acc: 0.0000e+00
Epoch 3/10
1590/1590 [==============================] - 0s 64us/step - loss: nan - acc: 0.0000e+00
Epoch 4/10
1590/1590 [==============================] - 0s 58us/step - loss: nan - acc: 0.0000e+00
Epoch 5/10
1590/1590 [==============================] - 0s 47us/step - loss: nan - acc: 0.0000e+00
Epoch 6/10
1590/1590 [==============================] - 0s 50us/step - loss: nan - acc: 0.0000e+00
Epoch 7/10
1590/1590 [==============================] - 0s 48us/step - loss: nan - acc: 0.0000e+00
Epoch 8/10
1590/1590 [==============================] - 0s 38us/step - loss: nan - acc: 0.0000e+00
Epoch 9/10
1590/1590 [==============================] - 0s 37us/step - loss: nan - acc: 0.0000e+00
Epoch 10/10
1590/1590 [==============================] - 0s 40us/step - loss: nan - acc: 0.0000e+00


In [55]:
evaluation = model.evaluate(X_test, y_test)
print('Q 6C::','Accuracy (Test Set) ', evaluation[1])

398/398 [==============================] - 0s 260us/step
Q 6C:: Accuracy (Test Set)  0.0


***
### Question 7###
Combine the features from question 4, 5, and exercise hours from question 6 (using the best normalization feature set form questions 5 and 6)

**Question 7.a**  
What was the NN accuracy on the test set using the single 10 node hidden layer?

In [61]:
avg = df_train['height'].mean()
std_deviation = df_train['height'].std()

transform_z_score = lambda x: (x - avg) / std_deviation

X_train = df_train[['eyecolor','year', 'exercise', 'height']]
X_train['height'] = X_train['height'].apply(transform_z_score)
X_train = X_train.to_dict('records')

#'exercisehours',

X_test = df_test[['eyecolor','year', 'exercise', 'height']]
X_test['height'] = X_test['height'].apply(transform_z_score)
X_test = X_test.to_dict('records')

vec = DictVectorizer()
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.fit_transform(X_test).toarray()

model = Sequential()
model.add(Dense(units=10, activation='sigmoid', input_dim=14))
model.add(Dense(units=1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
1590/1590 [==============================] - 0s 314us/step - loss: 0.6868 - acc: 0.4698
Epoch 2/50
1590/1590 [==============================] - 0s 47us/step - loss: 0.6552 - acc: 0.5786
Epoch 3/50
1590/1590 [==============================] - 0s 47us/step - loss: 0.6406 - acc: 0.7635
Epoch 4/50
1590/1590 [==============================] - 0s 49us/step - loss: 0.6320 - acc: 0.8031
Epoch 5/50
1590/1590 [==============================] - 0s 46us/step - loss: 0.6260 - acc: 0.8025
Epoch 6/50
1590/1590 [==============================] - 0s 49us/step - loss: 0.6208 - acc: 0.7881
Epoch 7/50
1590/1590 [==============================] - 0s 48us/step - loss: 0.6157 - acc: 0.7748
Epoch 8/50
1590/1590 [==============================] - 0s 47us/step - loss: 0.6107 - acc: 0.7799
Epoch 9/50
1590/1590 [==============================] - 0s 50us/step - loss: 0.6056 - acc: 0.7811
Epoch 10/50
1590/1590 [==============================] - 0s 48us/step - loss: 0.6006 - acc: 0.7811
Epoch 11/50
1590/1

In [62]:
evaluation = model.evaluate(X_test, y_test)
print('Q 7A::','Accuracy (Test Set) ', evaluation[1])

398/398 [==============================] - 0s 379us/step
Q 7A:: Accuracy (Test Set)  0.8492462305567372


**Question 7.b**  
What was the SVM accuracy on the test set the RBF kernel?

In [63]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test)
print('Q 7B::','Accuracy (Test Set) ', accuracy)

Q 7B:: Accuracy (Test Set)  0.8542713567839196


***
### Question 8- Bonus###
Can you improve your test set prediction accuracy by 5% or more?  

See how close to that milestone of improvement you can get by modifying the tuning parameters of either Neural Networks (the number of hidden layers, number of hidden nodes in each layer, the learning rate aka mu) or with SVM (choosing kernel, C, and gamma). A great guide to tuning parameters is explained in this guide: http://www.csie.ntu.edu.tw/~cjlin/papers/guide/guide.pdf. 

While the guide is specific to SVM and in particular the C and gamma parameters of the RBF kernel, the method applies to generally to any ML technique with tuning parameters.

Please also write a paragraph in a markdown cell below with an explanation of your approach and evaluation metrics.
